In [ ]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats

# Apartat (C): Analitzant Dades

1. Quin és el tipus de cada atribut? 
2. Quins atributs tenen una distribució Guassiana?
3. Quin és l'atribut objectiu? Per què?

In [ ]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('Admission_Predict_Ver1.1.csv')
data = dataset.values
labels = dataset.columns.values

1. Quin és el tipus de cada atribut? 

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

<style>
table {float:left}
</style>

| Atribut | Tipus de dada | Descripció |
| :-- | :-: | :-- |
|Serial No.|Nombre enter (int)| Posició de la entrada en ordre.|
|GRE Score|Nombre enter (int)| Puntuació del estudiant a els Graduate Record Examinations, els valors van desde 260 fins a 340, representant el sistema de puntuació a India.|
|TOEFL Score|Nombre enter (int)|Puntuació del estudiant a els Test Of English as a Foreign Language, els valors poden anar desde 0 fins a 120.|
|University Rating|Nombre enter (int)| Puntuació de la universitat pot variar entre 1 i 5.|
|SOP (Statement of Purpose)|Nombre Decimal (float)|Valoració de la qualitat del Statement of Purpose, en valors de 0 a 5 en intervals de 0’5.|
|LOR (Letter of Recommendation)|Nombre Decimal (float)|Valoració de la qualitat del Letter of Recommendation, en valors de 0 a 5 en intervals de 0’5.|
|CGPA (Undergraduate GPA)|Nombre Decimal (float)|Nota mitja de l'estudiant en els seus estudis undergrad. Va de 0 a 10.|
|Research Experience|Booleà (bool)|Representa si l’estudiant ha tingut experiència de recerca anteriorment on no.|
|Chance of Admission|Nombre Decimal (float)|Probabilitat de ser admès a la universitat donats tots els altres factors.|



2. Quins atributs tenen una distribució Guassiana?

In [ ]:
dataset.hist()

Els atributs que tenen una distribució gaussiana, basant en les grafiques son: GRE Score, TOEFL Score, University Rating, SOP, LOR, CGPA i Chance of Admission.

3. Quin és l'atribut objectiu? Per què?

L’atribut objectiu és “Chance of Admission”, ja que tots els atributs anteriors contribueixen a l'obtenció d’aquest. A més, és el més rellevant de predir: un estudiant preuniversitari de l'Índia que conegués el seu expedient acadèmic i la valoració de la seva universitat de preferència, segurament estarà interessat a calcular les seves probabilitats de ser admès.

# Apartat (B): Primeres regressions

1. Quin són els atributs més importants per fer una bona predicció?

2. Amb quin atribut s'assoleix un MSE menor?

3. Quina correlació hi ha entre els atributs de la vostra base de dades?

4. Com influeix la normalització en la regressió?

5. Com millora la regressió quan es filtren aquells atributs de les mostres que no contenen informació?

6. Si s'aplica un PCA, a quants components es redueix l'espai? Per què?

In [ ]:
def plan_split(size, precentatges):
    assert(sum(precentatges) <= 1)
    indices = np.arange(size)
    split = np.zeros(size)
    np.random.shuffle(indices)
    used = 0
    for i, p in enumerate(precentatges):
        n = int(np.floor(size * p))
        split[indices[used:n]] = i + 1
        used += n
    return split

def perform_split(X, y, split):
    X_ = []
    y_ = []
    for i in range(2):
        X_.append(X[split == i])
        y_.append(y[split == i])
    return X_, y_
    

X = data[:, [i for i in range(8)]]
y = data[:, 8]
split_info = plan_split(X.shape[0], [.15, .15])

X_, y_ = perform_split(X, y, split_info)

1. Quin són els atributs més importants per fer una bona predicció?

In [ ]:
import seaborn as sns
relacio = sns.pairplot(dataset)

In [ ]:
for i in range(8):
    sns.lmplot(x=labels[i], y=labels[8], data=dataset)

Els millors són *GRE Score*, *TOEFL Score* i *CGPA*.

2. Amb quin atribut s'assoleix un MSE menor?

In [ ]:
def chartMe(content,colnames,rownames,precision=9,pre="", extra=""):
    #Making the rows of the chart
    chart=[]
    for i,valuesRow in enumerate(content):
        newRow=[str(pre+" "+rownames[i]+" "+extra)]
        newRow.extend(valuesRow)
        chart.append(newRow)
        
    #Setting up the precision, as requested by function call
    pd.set_option('precision', precision)
    seriousChart=pd.DataFrame(chart, columns=colnames).style.hide_index()
    pd.reset_option('precision')
    
    return seriousChart

In [ ]:
chartMe(content=['a','b','c'],colnames=['attribute','value'],rownames=['row 1','row 2','row 3'])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as R2

def regression(x, y):
    # Creem un objecte de regressió de sklearn
    regr = LinearRegression()

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

def MSE(v1, v2):
    return float(((v1 - v2)**2).mean())

def calc_MSE(X, y):
    mse = []
    for i in range(X[0].shape[1]):
        r = regression(X[0][:,[i]], y[0])
        mse.append(MSE(r.predict(X[1][:,[i]]), y[1]))
    return mse

def calc_full(X, y):
    r = regression(X[0], y[0])
    pred = r.predict(X[1])
    return (MSE(pred, y[1]), R2(pred, y[1]))

def show_MSE(X, y, labels, extra=""):
    mse = calc_MSE(X, y)
    for i, val in enumerate(mse):
        print("MSE of ", labels[i], " ", extra, ": ", val, sep="")
        
def chart_MSE(X, y, labels, extra="", pre=""):
    mse = calc_MSE(X, y)

    chart=[]
    for i, val in enumerate(mse):
        newLineText=str(pre+" "+labels[i]+" "+extra)
        chart.append([newLineText , val])
    pd.options.display.float_format = "{:,.9f}".format
    
    chartDF=pd.DataFrame(chart, columns=["Atribute","MSE"]).style.hide_index()
    return chartDF

In [ ]:
print("\nL'atribut amb MSE menor és ", labels[mse.index(min(mse))], ".")
c=chart_MSE(X_, y_, labels)
c

L'atribut amb MSE menor és  CGPA .

3. Quina correlació hi ha entre els atributs de la vostra base de dades?

Les correlacions entre els atributs de la nostra base de dades es poden visualitzar amb el següent heatmap:

In [ ]:
correlacio = dataset.corr()

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)

Com ja suposàvem, el primer atribut ('Serial No.') no té correlació amb cap altre, ja que només indica la posició de cada instància al dataset (exemple: la primera instància té 'Serial No.'=1, la segona en té 2, l'enèsima en té 'n'). Per tant, seria de sentit comú ignorar aquesta dada completament a l'hora de continuar analitzant.

Per altra banda, les correlacions superiors o iguals al 75% són les següents:
- GRE Score vs TOEFL Score (0.83)
- GRE Score vs CGPA (0.83)
- GRE Score vs Chance of Admit (0.81)
- TOEFL vs CGPA (0.81)
- TOEFL vs Chance of Admit (0.79)
- CGPA vs Chance of Admit (0.88)

Però, com que la variable objectiu és Chance of Admit, de la llista anterior només ens interessarem pels elements tercer, cinquè i sisè. 

4. Com influeix la normalització en la regressió?

In [ ]:
def normalitzar(X):
    mean = X.mean(0)
    std = X.std(0)
    Xnorm = (X - mean[None, :]) / std[None, :]
    return Xnorm

Xnorm = normalitzar(X)
X_norm, y_norm = perform_split(Xnorm, y, split_info)

In [ ]:
print("\nL'atribut normalitzat amb MSE menor és", labels[mse_norm.index(min(mse_norm))])
chart_MSE(X_norm, y_norm, labels, "normalitzat")

In [ ]:
tol = 10**-12
dif = [e - e_norm if abs(e - e_norm) > tol else 0 for e, e_norm in zip(mse, mse_norm)]
a=chartMe(dif,rownames=labels,colnames=["Attribute", "Value"], pre="Difference of")
a

In [ ]:
no_normal = calc_full(X_, y_)
normal = calc_full(X_norm, y_norm)
chartMe(content=[no_normal,normal],rownames=["Sense normalitzar","Normalitzant"],colnames=["","MSE","$R^2$"],precision=19)

No hi ha cap canvi apreciable en cap de les mesures.

5. Com millora la regressió quan es filtren aquells atributs de les mostres que no contenen informació?

In [ ]:
Xinfo = Xnorm[:,[1, 2, 3, 4, 5, 6, 7]]
X_info, y_info = perform_split(Xinfo, y, split_info)

In [ ]:
all_used = calc_full(X_, y_)
only_info = calc_full(X_info, y_info)
chartMe(content=[all_used,
                 only_info],
        rownames=["Amb tots els atributs",
                  "Només amb l'informació útil"],
        colnames=["","MSE","$R^2$"])

Es pot veure una pujada (de l'ordre de $10^{-4}$, és a dir, insignificant) a l'MSE, i una baixada al $R^2$ de l'ordre de $10^{-2}$, és a dir, poc apreciable.

6. Si s'aplica un PCA, a quants components es redueix l'espai? Per què?

In [ ]:
from sklearn.decomposition import PCA

XbestPCA = PCA(n_components='mle').fit_transform(Xnorm)
X_bestPCA, y_bestPCA = perform_split(XbestPCA, y, split_info)

print("Espai reduit a", XbestPCA.shape[1], "dimensions.")

In [ ]:
all_used = calc_full(X_, y_)
only_info = calc_full(X_info, y_info)
PCA_used = calc_full(X_bestPCA, y_bestPCA)

chartMe(content=[all_used,
                only_info,
                PCA_used],
       rownames=["Amb tots els atributs",
                "Només els que contenen informacio util",
                "Reduït utilitzant PCA"],
       colnames=["","MSE","$R^2$"],
       precision=19)

# Apartat (A): El descens del gradient  

1. Com influeixen tots els paràmetres en el procés de descens? Quins valors de learning rate convergeixen més ràpid a la solució òptima? Com influeix la inicialització del model en el resultat final? 

2. Quines funcions polinomials (de diferent grau, de diferents combinacions d'atributs, ...) heu escollit per ser apreses amb el vostre descens del gradient? quina ha donat el millor resultat (en error i rapidesa en convergència)?

3. Utilitzeu el regularitzador en la fòrmula de funció de cost i descens del gradient i proveu polinomis de diferent grau. Com afecta el valor del regularitzador?

3. Quina diferència (quantitativa i qualitativa) hi ha entre el vostre regressor i el de la llibreria ?

4. Té sentit el model (polinomial) trobat quan es visualitza sobre les dades? 

5. Ajuda la visualització a identificar aquelles mostres per a les que el regressor obté els pitjors resultats de predicció? 

$$J(w) = \frac{1}{2m} \left[ \sum^m_{i=1}(f(x^{i}; w) - y^{i})^2 + \lambda\sum_{j=1}^{n}(w_{j}^2) \right]$$

$$w_0 = w_0 - \alpha \frac{1}{m} \sum_{i=1}^{m}(f(x^{i}; w)-y^{i}) \cdot 1$$
$$w_j = w_j - \alpha \left[\frac{1}{m} \sum_{i=1}^{m}(f(x^{i}; w)-y^{i}) \cdot x_{j}^{i} - \frac{\lambda}{m}w_{j} \right]$$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

class Regressor(object):
    def __init__(self, alpha=0.3, regularitzador=0.0, initial_bias=None, initial_weight=None):
        if initial_bias == None:
            self.w0 = np.random.rand()
        else:
            self.w0 = initial_bias
        if initial_weight == None:
            self.wj = np.array([])
        else:
            self.wj = initial_weight
        self.alpha = alpha
        self.regularitzador = regularitzador
        self.loaded = False
    
    def predict(self, x):
        return self.w0 + x.dot(self.wj)
    
    def __cost(self, hy, y):
        return (np.square(hy - y).sum() + self.regularitzador * np.square(self.wj).sum()) / (2 * y.size)
    
    def __update(self, hy, y):
        self.w0 -= self.alpha * (hy - y).sum() / y.size
        self.wj -= self.alpha / y.size * ((hy - y).dot(self.X) - self.regularitzador * self.wj)
    
    def train(self, max_iter, epsilon=0.000001):
        if not self.loaded:
            raise ValueError("No dataset loaded for training")
        hy = self.predict(self.X)
        J = self.__cost(hy, self.y)
        for i in range(max_iter):
            self.__update(hy, self.y)
            hy = self.predict(self.X)
            Jnew = self.__cost(hy, self.y)
            delta = abs(Jnew - J)
            if delta < epsilon:
                break
            J = Jnew
    
    def load(self, Xtrain, ytrain):
        self.X = Xtrain
        self.y = ytrain
        if self.wj.size != self.X.shape[1]:
            self.wj = np.random.rand(self.X.shape[1])
        self.loaded = True

def make_polinomial(X, degree=1):
    poli = PolynomialFeatures(degree=degree, include_bias=False)
    return poli.fit_transform(X)

In [ ]:
# Generar Polinomi exemple
make_polinomial(Xnorm[:,[5]], degree=4)

In [ ]:
### EXEMPLES ###

# Regressor implementat per X_5 i X_5^2
X_poli1, y_poli1 = perform_split(make_polinomial(Xnorm[:,[5]], degree=2), y, split_info)
r = Regressor(alpha=.05, regularitzador=10)
r.load(X_poli1[0], y_poli1[0])
r.train(4000, epsilon=0.0001)
print(MSE(r.predict(X_poli1[0]), y_poli1[0]))
print(MSE(r.predict(X_poli1[1]), y_poli1[1]))

# Regressor implementat per X_5, X_5^2, X_5^3 i X_5^4
X_poli2, y_poli2 = perform_split(make_polinomial(Xnorm[:,[5]], degree=4), y, split_info)
r = Regressor(alpha=.01, regularitzador=0)
r.load(X_poli2[0], y_poli2[0])
r.train(4000, epsilon=0)
print(MSE(r.predict(X_poli2[0]), y_poli2[0]))
print(MSE(r.predict(X_poli2[1]), y_poli2[1]))

# Regressor importat per X_5 i X_5^2
r = regression(X_poli1[0], y_poli1[0])
print(MSE(r.predict(X_poli1[0]), y_poli1[0]))
print(MSE(r.predict(X_poli1[1]), y_poli1[1]))